In [5]:
import os
import numpy as np
from PIL import Image
from scipy.io import loadmat
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt


predicted_mask_dir = "masks"  
mask_ext = ".png" 

def load_mask(path):
   
    if path.endswith('.npy'):
        return np.load(path)
    elif path.endswith('.png'):
        return np.array(Image.open(path))
    elif path.endswith('.mat'):
        mat = loadmat(path)
       
        return mat['mask'].astype(np.uint8)
    else:
        raise ValueError("Unknown mask format")

def extract_features(mask):
   
    od = (mask == 1)
    oc = (mask == 2)
    od_area = np.sum(od)
    oc_area = np.sum(oc)
    rim_area = od_area - oc_area if od_area > oc_area else 0
    od_rows = np.any(od, axis=1)
    oc_rows = np.any(oc, axis=1)
    od_height = np.ptp(np.where(od_rows)[0]) + 1 if np.any(od_rows) else 0
    oc_height = np.ptp(np.where(oc_rows)[0]) + 1 if np.any(oc_rows) else 0
    vertical_cdr = oc_height / od_height if od_height > 0 else 0.0
    cdr_area = oc_area / od_area if od_area > 0 else 0.0
    return [cdr_area, vertical_cdr, od_area, oc_area, rim_area]

def pseudo_label_from_cdr(cdr, threshold=0.6):
    return int(cdr > threshold)


features, pseudo_labels = [], []
for fname in os.listdir(predicted_mask_dir):
    if not fname.endswith(mask_ext):
        continue
    mask = load_mask(os.path.join(predicted_mask_dir, fname))
    feats = extract_features(mask)
    features.append(feats)
    pseudo_labels.append(pseudo_label_from_cdr(feats[0])) 

features = np.array(features)
pseudo_labels = np.array(pseudo_labels)


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_probs, all_trues = [], []
for train_idx, test_idx in skf.split(features, pseudo_labels):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = pseudo_labels[train_idx], pseudo_labels[test_idx]
    clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    clf.fit(X_train, y_train)
    probs = clf.predict_proba(X_test)[:, 1]  
    all_probs.extend(probs)
    all_trues.extend(y_test)
all_probs = np.array(all_probs)
all_trues = np.array(all_trues)



from sklearn.metrics import accuracy_score, roc_auc_score

y_pred = (all_probs >= 0.5).astype(int)


acc = accuracy_score(all_trues, y_pred)


print(f'Predicted Mask Classification -- Accuracy: {acc:.4f}, , Threshold: 0.500')




Predicted Mask Classification -- Accuracy: 0.9969, , Threshold: 0.500
